# 1. Acceder a la data usando método Flight

In [3]:
from ibm_watson_studio_lib import access_project_or_space
import os
wslib = access_project_or_space()
db2_connection = wslib.get_connection('db2_connection_unmasked')
wslib.show(db2_connection)

{'database': 'BUDA',
 'password': '{"secretRefID":"1000331005:db2-secret","key":"password"}',
 'username_password_security': 'clear_text',
 'port': '25010',
 'vaulted_properties': 'password,username',
 'host': '9.30.182.153',
 'username_password_encryption': 'default',
 'ssl': 'false',
 'inherit_access_token': 'false',
 'username': '{"secretRefID":"1000331005:db2-secret","key":"username"}',
 '.': {'name': 'db2_connection_unmasked',
       'description': None,
       'asset_id': 'connection.db2connection',
       'asset_type': 'connection',
       'personal_credentials': True}}


In [4]:
import itc_utils.flight_service as itcfs
from IPython.display import display, HTML
nb_data_request = {
'connection_name': """db2_connection_unmasked""",
'interaction_properties': {
'select_statement': 'SELECT * FROM "DB2INST1"."CUSTOMER" FETCH FIRST 5000 ROWS ONLY'
}
}
flight_descriptor = itcfs.get_flight_descriptor(nb_data_request=nb_data_request)
flightClient = itcfs.get_flight_client()
flightInfo = flightClient.get_flight_info(flight_descriptor)
df_0 = itcfs.read_pandas_and_concat(flightClient, flightInfo, timeout=240)
df_0.head(10)

,CUSTOMER_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,ADDRESS,CREATED_AT
0,1,John,Doe,john.doe@example.com,1234567890,123 Elm Street,2025-01-20 09:41:46.112032
1,2,Jane,Smith,jane.smith@example.com,0987654321,456 Oak Avenue,2025-01-20 09:41:46.112032
2,3,Michael,Johnson,michael.johnson@example.com,None,789 Pine Road,2025-01-20 09:41:46.112032
3,4,Alice,Brown,alice.brown@example.com,1112223333,101 Maple Street,2025-01-20 09:42:23.452694
4,5,Bob,Davis,bob.davis@example.com,2223334444,202 Birch Lane,2025-01-20 09:42:23.452694
5,6,Charlie,Evans,charlie.evans@example.com,3334445555,303 Cedar Court,2025-01-20 09:42:23.452694
6,7,Diana,Wilson,diana.wilson@example.com,4445556666,404 Ash Drive,2025-01-20 09:42:23.452694
7,8,Ethan,Taylor,ethan.taylor@example.com,5556667777,505 Fir Road,2025-01-20 09:42:23.452694
8,9,Fiona,Anderson,fiona.anderson@example.com,6667778888,606 Spruce Street,2025-01-20 09:42:23.452694
9,10,George,Harris,george.harris@example.com,7778889999,707 Walnut Avenue,2025-01-20 09:42:23.452694


# 2. Acceder a la data usando API documentada de Secrets

Primero obtenemos identificador del secreto que buscamos

In [35]:
import requests
import os
cpd_cluster_host = "cpd-cpd.apps.cp4d-education-install.cp.fyre.ibm.com" # Reemplazar host
token = os.environ['USER_ACCESS_TOKEN']
url = f"https://{cpd_cluster_host}/zen-data/v2/secrets"
headers = {
"Accept": "application/json",
"Authorization": f"Bearer {token}" # También soporta ZenApiKey
}
params = {
"limit": 2,
"sort": "updated_at",
}
response = requests.get(url, headers=headers, params=params, verify=False)
search_secret_name = "db2-secret"
secrets = response.json().get("secrets", [])
secret_urn = {secret["secret_name"]: secret["secret_urn"] for secret in secrets}
print(response.status_code)
print(secret_urn)
# print(response.json())

200
{'db2-secret': '1000331005:db2-secret'}


Luego obtenemos detalles del secreto

In [49]:
url = f"https://{cpd_cluster_host}/zen-data/v2/secrets/{secret_urn.get(search_secret_name)}"
headers = {
"Accept": "application/json",
"Authorization": f"Bearer {token}", # También soporta ZenApiKey
"exclude_secret_data": "False"
}
response = requests.get(url, headers=headers, params=params, verify=False)
print(response.status_code)
print(response.json())
credentials = response.json().get('data', {}).get('secret', {}).get('credentials', {})
username = credentials.get('username', 'Unknown')
password = credentials.get('password', 'Unknown')
print(f"el usuario es: {username}")
print(f"la contraseña del usuario es: {password}")

200
{'data': {'secret': {'credentials': {'password': 'demobancogalicia123', 'username': 'admin'}}}, 'metadata': {'created_at': '2025-01-29T01:21:32.182154Z', 'created_by': 'jlara', 'owner_uid': '1000331005', 'secret_name': 'db2-secret', 'secret_urn': '1000331005:db2-secret', 'type': 'credentials', 'updated_at': '2025-01-29T02:30:07.517797Z', 'vault_name': 'internal', 'vault_urn': '0000000000:internal'}}
el usuario es: admin
la contraseña del usuario es: demobancogalicia123


# 3. Acceder a la data usando libreria interna de CP4D

In [6]:
token =
import icpd_core, json
from icpd_core import icpd_util
my_secrets = icpd_util.get_my_secrets_v2()
print(my_secrets)
my_test_secret = icpd_util.get_my_secret_details_v2("1000331001:test-secret")
my_test_secret_creds = my_test_secret['data']['secret']['credentials']
my_db_username = my_test_secret_creds['username']
my_db_password = my_test_secret_creds['password']
print(my_db_username)
print(my_db_password)

[{'created_at': '2025-01-29T01:21:32.182154Z', 'created_by': 'jlara', 'invalid_auth': 'false', 'owner_uid': '1000331005', 'secret_name': 'db2-secret', 'secret_urn': '1000331005:db2-secret', 'type': 'credentials', 'updated_at': '2025-01-29T01:21:32.182154Z', 'vault_name': 'internal', 'vault_urn': '0000000000:internal'}]
